# MNIST Pytorch Classification Sample Code

2020/05/01, Andy RK Chang v1.0 initial 

2020/05/02, Andy RK Chang v2.0 revise parameter



MNIST interactive examples in pytorch and tensorflow

https://github.com/pyliaorachel/MNIST-pytorch-tensorflow-eager-interactive


In [0]:
# Read file from Google Drive

from google.colab import drive
import os
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Build Model

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms    # torchvision contains common utilities for computer vision

In [0]:
# Class : model definition 

class Net(nn.Module):  # Inherit from `nn.Module`, define `__init__` & `forward`
    def __init__(self):
        # Always call the init function of the parent class `nn.Module`
        # so that magics can be set up.
        super(Net, self).__init__()

        # Define the parameters in your network.
        # This is achieved by defining the shapes of the multiple layers in the network.

        # Define two 2D convolutional layers (1 x 10, 10 x 20 each)
        # with convolution kernel of size (5 x 5).
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        # Define a dropout layer
        self.conv2_drop = nn.Dropout2d()

        # Define a fully-connected layer (320 x 10)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        # Define the network architecture.
        # This is achieved by defining how the network forward propagates your inputs

        # Input image size: 28 x 28, input channel: 1, batch size (training): 64 

        # Input (64 x 1 x 28 x 28) -> Conv1 (64 x 10 x 24 x 24) -> Max Pooling (64 x 10 x 12 x 12) -> ReLU -> ...
        x = F.relu(F.max_pool2d(self.conv1(x), 2))

        # ... -> Conv2 (64 x 20 x 8 x 8) -> Dropout -> Max Pooling (64 x 20 x 4 x 4) -> ReLU -> ...
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))

        # ... -> Flatten (64 x 320) -> ...
        x = x.view(-1, 320)

        # ... -> FC (64 x 10) -> ...
        x = self.fc(x)

        # ... -> Log Softmax -> Output
        return F.log_softmax(x, dim=1)

# Model Training

In [0]:
#
# Modified from PyTorch examples: 
# https://github.com/pytorch/examples/blob/master/mnist/main.py
#

import os
import argparse
#import torch
#import torch.nn as nn
#import torch.nn.functional as F
#import torch.optim as optim
#from torchvision import datasets, transforms    # torchvision contains common utilities for computer vision
from torch.autograd import Variable
from argparse import ArgumentParser

#from .model import Net 


In [0]:
# Function Module

def load_data(train_batch_size, test_batch_size):
    """Fetch MNIST dataset

    MNIST dataset has built-in utilities set up in the `torchvision` package, so we just use the `torchvision.datasets.MNIST` module (http://pytorch.org/docs/master/torchvision/datasets.html#torchvision.datasets.MNIST) to make our lives easier.
    """

    kwargs = {} 

    # Fetch training data
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True, **kwargs)

    # Fetch test data
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

    return (train_loader, test_loader)

def train(model, optimizer, epoch, train_loader, log_interval):
    # State that you are training the model
    model.train()

    # Iterate over batches of data
    for batch_idx, (data, target) in enumerate(train_loader):
        # Wrap the input and target output in the `Variable` wrapper
        data, target = Variable(data), Variable(target)

        # Clear the gradients, since PyTorch accumulates them
        optimizer.zero_grad()

        # Forward propagation
        output = model(data)

        # Calculate negative log likelihood loss
        loss = F.nll_loss(output, target)

        # Backward propagation
        loss.backward()

        # Update the gradients
        optimizer.step()

        # Output debug message
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def test(model, test_loader):
    # State that you are testing the model; this prevents layers e.g. Dropout to take effect
    model.eval()

    # Init loss & correct prediction accumulators
    test_loss = 0
    correct = 0

    # Optimize the validation process with `torch.no_grad()`
    with torch.no_grad():
        # Iterate over data
        for data, target in test_loader: # Under `torch.no_grad()`, no need to wrap data & target in `Variable`
            # Retrieve output
            output = model(data)

            # Calculate & accumulate loss
            test_loss += F.nll_loss(output, target, reduction='sum').data.item()

            # Get the index of the max log-probability (the predicted output label)
            pred = output.data.argmax(1)

            # If correct, increment correct prediction accumulator
            correct += pred.eq(target.data).sum()

    # Print out average test loss
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [0]:
# Instantiate the model
model = Net()

#args =  Namespace(batch_size=64, epochs=10, log_interval=10, lr=0.01, momentum=0.5, seed=1, test_batch_size=1000)

parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',\
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',\
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',\
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',\
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',\
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--seed', type=int, default=1, metavar='S',\
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',\
                    help='how many batches to wait before logging training status')

#args = parser.parse_args()   # for command line
args = parser.parse_args(['--batch-size=64', '--test-batch-siz=1000', '--epochs=10', '--lr=0.01', '--momentum=0.5', '--seed=1', '--log-interval=10'])


# Choose SGD as the optimizer, initialize it with the parameters & settings
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

# Load data
train_loader, test_loader = load_data(args.batch_size, args.test_batch_size)

# Train & test the model
for epoch in range(1, args.epochs + 1):
     train(model, optimizer, epoch, train_loader, log_interval=args.log_interval)
     test(model, test_loader)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.416496
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.284373
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.152807
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.015532
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.892992
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.519460
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.371071
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.170158
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.778500
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.889671
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.727126
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.626816
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.682499
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.556994
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.571436
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.410163
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.451886
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.511347
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.440953
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.548946
Train Epoch: 1 [12800/60000 (

In [0]:
# Save the model for future use
# train & test model - on python command line
# $ python -m pytorch.train

__file__ = '/content/gdrive/My Drive/Colab Notebooks/MNIST/Pytorch/pytorch/train.py'
package_dir = os.path.dirname(os.path.abspath(__file__))
model_path = os.path.join(package_dir,'model')
torch.save(model.state_dict(), model_path)

# Predicitor

In [0]:
#
# Handwritten number predictor
#

import os
import argparse
from PIL import Image
import torch
from torchvision import transforms

#from .model import Net

"""Settings"""

# test model interactively on python command line
# $ python -m pytorch.app --image=<path-to-image>

__file__ = '/content/gdrive/My Drive/Colab Notebooks/MNIST/Pytorch/pytorch/app.py'

package_dir = os.path.dirname(os.path.abspath(__file__))
default_img_path = os.path.join(package_dir,'test_2.png')

parser = argparse.ArgumentParser(description='PyTorch MNIST Predictor')
parser.add_argument('--image', type=str, default=default_img_path, metavar='IMG',
                            help='image for prediction (default: {})'.format(default_img_path))

#args = parser.parse_args() # for python command line using
args = parser.parse_args(['--image=' + os.path.join(package_dir,'test_4.png')])


"""Make Prediction"""

# Load model
model_path = os.path.join(package_dir,'model')
model = Net()
model.load_state_dict(torch.load(model_path))

# Load & transform image
ori_img = Image.open(args.image).convert('L')
t = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
img = torch.autograd.Variable(t(ori_img).unsqueeze(0))
ori_img.close()

# Predict
model.eval()
output = model(img)
pred = output.data.max(1, keepdim=True)[1][0][0]
print('Prediction: {}'.format(pred))

/content/gdrive/My Drive/Colab Notebooks/MNIST/Pytorch/pytorch/test_2.png
Prediction: 4
